In [2]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [3]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')
train_data_lsa_cvjd_54339

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [5]:
train_data_lsa_cvjd_54339.reset_index(drop=True)

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [6]:
pd.read_pickle('../generalization_data/train_description_tfidf_lsa.pkl')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.245978,0.219388,-0.001255,0.060506,-0.019606,-0.018927,-0.069812,0.049427,-0.047108,0.038293,...,-0.002675,0.021072,-0.007054,-0.020001,-0.038634,0.009341,-0.027581,-0.035102,0.035068,-0.001629
1,0.263932,0.104354,-0.086183,-0.067204,0.090541,0.068914,-0.058441,0.039959,-0.050831,0.062856,...,0.045697,0.020841,-0.027177,-0.010975,0.027781,0.000108,0.040334,0.004524,0.002817,0.001695
2,0.276421,-0.027352,0.082342,0.081769,-0.026448,0.040152,-0.166858,-0.146736,-0.006864,0.022844,...,0.019581,0.007622,0.062413,-0.046397,-0.013292,-0.008556,0.009356,0.005523,-0.017453,0.012941
3,0.203287,-0.005198,-0.064190,0.127608,0.017330,-0.101731,0.151955,-0.042419,-0.012835,0.016198,...,0.191235,-0.022779,-0.006482,0.036324,0.056972,0.012149,-0.053477,-0.086235,0.046087,0.049085
4,0.204987,-0.006300,-0.064197,0.126069,0.016452,-0.101222,0.151119,-0.042060,-0.013972,0.017858,...,0.186252,-0.018217,0.003790,0.035659,0.052471,0.010645,-0.045802,-0.081005,0.049206,0.047114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,0.262550,-0.085161,0.077125,0.071276,-0.005350,-0.078712,0.019407,-0.024380,-0.076040,0.089582,...,0.074208,0.037490,0.033920,-0.046867,-0.030337,-0.060884,0.018400,0.027278,0.025553,0.011606
54335,0.134888,-0.043534,0.027936,0.016889,-0.039029,0.016183,-0.029966,-0.051795,0.005815,0.001073,...,0.009807,-0.022126,-0.006157,0.007902,-0.029644,0.024964,0.001999,-0.012915,-0.048732,0.015322
54336,0.192223,0.250674,0.053166,-0.120096,-0.116748,-0.052939,0.113171,0.097601,-0.007018,0.012832,...,-0.014897,-0.004287,-0.028053,-0.001634,-0.023137,0.006049,-0.016478,-0.055726,0.028070,0.016210
54337,0.255035,-0.094685,-0.170121,-0.007023,-0.024598,-0.049759,0.043359,-0.068628,-0.007909,-0.002515,...,-0.016107,-0.024316,0.004395,0.022126,0.002917,-0.027468,-0.025804,-0.049746,0.019801,-0.036044


In [7]:
# 读取生成的5份tfidf_pca文件，并拼接
# 注意，不是join的时候，concat是通过index拼接的。所以需要将索引重置。
sample = train_data_lsa_cvjd_54339.reset_index(drop=True)

# 需要将这个向量变成一个列表
def tfidf_lsa_merge_fun(series):
    return list(series)

tfidf_lsa_file = ['train_description_tfidf_lsa.pkl', 'train_position_tfidf_lsa.pkl', 'train_requirement_tfidf_lsa.pkl', 'train_skills_tfidf_lsa.pkl', 'train_title_category_tags_tfidf_lsa.pkl']
for pkl_file in tfidf_lsa_file:
    tfidf_lsa = pd.read_pickle('../generalization_data/'+pkl_file)
    print(pkl_file)
    #print(tfidf_pca)
    col_name = pkl_file.strip('.pkl') # 把skills的s去掉了
    tfidf_lsa[col_name] = tfidf_lsa.apply(tfidf_lsa_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_lsa[[col_name]]], axis=1)

tfidf_lsa_col_names = [pkl_file.strip('.pkl') for pkl_file in tfidf_lsa_file]
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]

train_description_tfidf_lsa.pkl
train_position_tfidf_lsa.pkl
train_requirement_tfidf_lsa.pkl
train_skills_tfidf_lsa.pkl
train_title_category_tags_tfidf_lsa.pkl


,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.24597831805245932, 0.21938832094389843, -0....","[0.007319538298968588, 0.00033543767052876544,...","[0.03301313771162994, -0.03568899315863845, -0...","[0.5376593767920079, -0.14639548336625344, -0....","[0.13961902921276909, -0.030086475656111354, -..."
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201544531714, 0.10435439925841314, -0....","[0.09957450441008547, -0.02088326629016182, -0...","[0.37018351064663224, -0.20255127007668178, -0...","[0.2360612645429957, -0.12857913684524475, 0.1...","[0.5419246758199077, -0.18238693292544703, -0...."
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059836046595, -0.027351697795842202, 0...","[0.04322279521136983, 0.011083099764111395, 0....","[0.2894134101949335, -0.047410431459986115, -0...","[0.004060041402042113, 0.005098857023259797, 0...","[0.09166408309186733, 0.013460338908480717, -0..."
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670870610283, -0.005197812926017977, -...","[0.15026255431619845, -0.0049050966470799914, ...","[0.1861817383295255, 0.03116853921171071, 0.04...","[0.15559348203332388, 0.15537462565673107, -0....","[0.0878508783508848, 0.045965498421162765, -0...."
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743458236648, -0.006300228929310872, -...","[0.15026255431619845, -0.0049050966470799914, ...","[0.1916301156536554, 0.025951257082786224, 0.0...","[0.15559348203332388, 0.15537462565673107, -0....","[0.10718260997232798, 0.055657615802857136, -0..."
...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502867829111, -0.08516050914120489, 0.0...","[0.23530919206618117, 0.79447673276088, 0.0180...","[0.3028272621469679, 0.1609754720857342, 0.006...","[0.27210694685311315, 0.4244178618054231, -0.0...","[0.08847076546570108, 0.22734134025635327, -0...."
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778903066007, -0.04353380621631703, 0....","[0.6087412978467188, -0.14738610294678545, -0....","[0.15424621064685295, -0.0012759694617245497, ...","[0.26284875473293323, 0.007522235306986733, -0...","[0.19044429490421788, -0.019572546832142282, 0..."
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.19222344357907623, 0.25067411001735296, 0.0...","[0.024225992623860555, -0.0015968739885410417,...","[0.2357303760769574, -0.13812539531088575, 0.1...","[0.35264789501603516, -0.208997006408272, 0.11...","[0.11713308547264276, -0.006170363421938364, -..."
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351131500704, -0.09468525459793202, -0....","[0.8678408194440407, -0.22318129488049962, -0....","[0.1901935716954906, -0.12296863457979303, -0....","[0.27948075976837006, -0.04971021544977418, 0....","[0.3466942283492569, -0.07639249803035937, 0.3..."


In [8]:
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names].to_pickle('../generalization_data/train_data_lsa_cvjd_merge_54339.pkl')